# Data Quality Checks

Notebook для ручной валидации выгруженных свечей: проверяем покрытие по дням, пропуски и сравниваем с автогенерируемым отчётом (`backend/scripts/monitoring/data_quality_report.py`).

In [ ]:
from datetime import datetime, time, timezone

import pandas as pd

from backend.scripts.monitoring.data_quality_report import load_candles, evaluate_quality

SECIDS = ["SBER", "GAZP", "LKOH", "GMKN"]
TIMEFRAME = "1m"
START = datetime(2024, 10, 1, tzinfo=timezone.utc)
END = datetime(2024, 11, 15, 23, 59, 59, tzinfo=timezone.utc)

df = load_candles(SECIDS, TIMEFRAME, START, END)
df.head()

In [ ]:
summary = evaluate_quality(df, TIMEFRAME, START, END)
summary["text_summary"], summary["secids"]["SBER"]

In [ ]:
per_day = df.assign(day=df["timestamp"].dt.date).groupby(["day", "secid"]).size().unstack("secid", fill_value=0)
per_day.tail()

In [ ]:
import json
from pathlib import Path

report_path = Path("docs/data_quality/candles_1m_2024-10-01_2024-11-15.json")
json.loads(report_path.read_text(encoding="utf-8"))